In [1]:
import paddle.fluid as fluid
class_num = 7
x = fluid.layers.data(name='x', shape=[3, 10], dtype='float32') # -1 3 10
label = fluid.layers.data(name='label', shape=[1], dtype='int64') #-1 1
predict = fluid.layers.fc(input=x, size=class_num, act='softmax')#-1 7
cost = fluid.layers.cross_entropy(input=predict, label=label)
print(x.shape,label.shape,predict.shape,cost)
print(x)

(-1, 3, 10) (-1, 1) (-1, 7) name: "cross_entropy2_0.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 1
    }
    lod_level: 0
  }
}
persistable: false

name: "x"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 3
      dims: 10
    }
    lod_level: 0
  }
}
persistable: false



In [109]:
import paddle

inputs = paddle.rand((256, 64))

linear = paddle.nn.Linear(64, 8, bias_attr=False)
loss_fn = paddle.nn.MSELoss()
optimizer = paddle.optimizer.Adam(0.01, parameters=linear.parameters())

for i in range(10):
    hidden = linear(inputs)
    # weight from input to hidden is shared with the linear mapping from hidden to output
    outputs = paddle.matmul(hidden, linear.weight, transpose_y=True)
    loss = loss_fn(outputs, inputs)
    loss.backward()
    print("step: {}, loss: {}".format(i, loss.numpy()))
    optimizer.step()
    optimizer.clear_grad()

step: 0, loss: [0.35952604]
step: 1, loss: [0.31887382]
step: 2, loss: [0.2995552]
step: 3, loss: [0.28412756]
step: 4, loss: [0.26553214]
step: 5, loss: [0.24091512]
step: 6, loss: [0.21059522]
step: 7, loss: [0.17760354]
step: 8, loss: [0.14681032]
step: 9, loss: [0.12299061]


In [32]:
import paddle
import paddle.nn.functional as F
from loguru import logger

logger.add('fc.log')

fc = paddle.nn.Linear(8,5)
logger.info('fc.weight:%s'%fc.weight)
logger.info('fc.bias:%s'%fc.bias)
# 设置优化器
optim = paddle.optimizer.Adam(learning_rate=0.001, parameters=fc.parameters())
x = paddle.rand([2,2,2,2])
label = paddle.to_tensor([1,0],dtype='int64')

x=paddle.reshape(x,[-1,8])
logger.info('x:%s'%x)
# 前向传播
predicts = fc(x)
logger.info('fc(x):%s'%predicts)
#predicts=F.softmax(predicts) # cross_entroy会计算softmax，所以这里不要计算
logger.info('predicts:%s'%predicts)
# 计算损失
loss = F.cross_entropy(predicts, label)
logger.info('loss:%s'%loss)
# 开始进行反向传播
loss.backward()
# 更新参数
optim.step()
loss.clear_grad()
logger.info(loss.numpy()[0])
'''
# 计算在cross_entry前计算softmax的计算结果，当然不应该计算，这里是验证，才发现不应该计算
2022-11-01 13:30:17.745 | INFO     | __main__:<module>:8 - fc.weight:Parameter containing:
Tensor(shape=[8, 5], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[-0.26917499,  0.21250401, -0.64733118, -0.55289793,  0.64664483],
        [ 0.03267508, -0.23102362, -0.62312615, -0.66298652,  0.56355345],
        [ 0.10465726,  0.14252797, -0.65502608,  0.30007410, -0.56954384],
        [ 0.14217915,  0.17066082, -0.02539300, -0.16849701, -0.14745426],
        [ 0.63527560,  0.15381630, -0.60549498, -0.15355670, -0.44909406],
        [ 0.35809821, -0.03604871,  0.42882413, -0.52067983,  0.09776447],
        [ 0.30929220,  0.09259517, -0.57030648,  0.01338258,  0.59317863],
        [-0.60106969,  0.13676916, -0.09769146,  0.49599123, -0.23148265]])
2022-11-01 13:30:17.748 | INFO     | __main__:<module>:9 - fc.bias:Parameter containing:
Tensor(shape=[5], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [0., 0., 0., 0., 0.])
2022-11-01 13:30:17.753 | INFO     | __main__:<module>:16 - x:Tensor(shape=[2, 8], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[0.78531247, 0.81736028, 0.79681695, 0.15102093, 0.93136758, 0.04332138, 0.16646625, 0.49245149],
        [0.12513840, 0.55602980, 0.11535818, 0.45463714, 0.78989142, 0.84746808, 0.13083205, 0.39368597]])
2022-11-01 13:30:17.757 | INFO     | __main__:<module>:19 - fc(x):Tensor(shape=[2, 5], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[ 0.28286302,  0.34185854, -2.23185253, -0.68153381,  0.06306744],
        [ 0.67030537,  0.14907347, -0.74252456, -0.84535497, -0.02387509]])
2022-11-01 13:30:17.761 | INFO     | __main__:<module>:21 - predicts:Tensor(shape=[2, 5], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[0.30070192, 0.31897572, 0.02432255, 0.11463151, 0.24136828],
        [0.39117682, 0.23227635, 0.09523331, 0.08592711, 0.19538642]])
2022-11-01 13:30:17.763 | INFO     | __main__:<module>:24 - loss:Tensor(shape=[1], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [1.46066070])
2022-11-01 13:30:17.766 | INFO     | __main__:<module>:30 - 1.4606607
'''

2022-11-01 13:30:17.745 | INFO     | __main__:<module>:8 - fc.weight:Parameter containing:
Tensor(shape=[8, 5], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[-0.26917499,  0.21250401, -0.64733118, -0.55289793,  0.64664483],
        [ 0.03267508, -0.23102362, -0.62312615, -0.66298652,  0.56355345],
        [ 0.10465726,  0.14252797, -0.65502608,  0.30007410, -0.56954384],
        [ 0.14217915,  0.17066082, -0.02539300, -0.16849701, -0.14745426],
        [ 0.63527560,  0.15381630, -0.60549498, -0.15355670, -0.44909406],
        [ 0.35809821, -0.03604871,  0.42882413, -0.52067983,  0.09776447],
        [ 0.30929220,  0.09259517, -0.57030648,  0.01338258,  0.59317863],
        [-0.60106969,  0.13676916, -0.09769146,  0.49599123, -0.23148265]])
2022-11-01 13:30:17.748 | INFO     | __main__:<module>:9 - fc.bias:Parameter containing:
Tensor(shape=[5], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [0., 0., 0., 0., 0.])
2022-11-01 13:30:17.753 | INFO     | __

In [108]:
import numpy as np
fc_w=np.array([[-0.26917499,  0.21250401, -0.64733118, -0.55289793,  0.64664483],
               [ 0.03267508, -0.23102362, -0.62312615, -0.66298652,  0.56355345],
               [ 0.10465726,  0.14252797, -0.65502608,  0.30007410, -0.56954384],
               [ 0.14217915,  0.17066082, -0.02539300, -0.16849701, -0.14745426],
               [ 0.63527560,  0.15381630, -0.60549498, -0.15355670, -0.44909406],
               [ 0.35809821, -0.03604871,  0.42882413, -0.52067983,  0.09776447],
               [ 0.30929220,  0.09259517, -0.57030648,  0.01338258,  0.59317863],
               [-0.60106969,  0.13676916, -0.09769146,  0.49599123, -0.23148265]])
fc_b=np.array([0., 0., 0., 0., 0.])
x=np.array([[0.78531247, 0.81736028, 0.79681695, 0.15102093, 0.93136758, 0.04332138, 0.16646625, 0.49245149],
            [0.12513840, 0.55602980, 0.11535818, 0.45463714, 0.78989142, 0.84746808, 0.13083205, 0.39368597]])
y=np.array([[ 0.28286302,  0.34185854, -2.23185253, -0.68153381,  0.06306744],
            [ 0.67030537,  0.14907347, -0.74252456, -0.84535497, -0.02387509]])
predicts_softmax=np.array([[0.30070192, 0.31897572, 0.02432255, 0.11463151, 0.24136828],
                   [0.39117682, 0.23227635, 0.09523331, 0.08592711, 0.19538642]])
loss=1.4606607
_y=np.matmul(x,fc_w)+fc_b
_y=np.exp(y)/np.sum(np.exp(y),axis=1,keepdims=True)
print(_y)
# cross_entroy会先计算softmax，然后再计算交叉熵，所以不用自己计算softmax
_y=np.exp(_y)/np.sum(np.exp(_y),axis=1,keepdims=True)
_y=np.log(_y)
print(_y)
labels=np.array([1,0])
#
print((-_y[0][1]-_y[1][0])/2)


[[0.30070193 0.31897573 0.02432255 0.1146315  0.24136829]
 [0.39117682 0.23227634 0.09523331 0.08592711 0.19538642]]
[[-1.51503367 -1.49675987 -1.79141305 -1.7011041  -1.57436732]
 [-1.42456139 -1.58346187 -1.7205049  -1.72981111 -1.62035179]]
1.4606606295728652


In [75]:
import paddle
import numpy as np

input_data = paddle.uniform([2, 5], dtype="float64")
label_data = np.random.randint(0, 2, size=(2)).astype(np.int64)


input =  paddle.to_tensor(input_data)
label =  paddle.to_tensor(label_data)
print(input)
print(label)

ce_loss = paddle.nn.CrossEntropyLoss()
output = ce_loss(input, label)
print(output)
'''
Tensor(shape=[2, 5], dtype=float64, place=CUDAPlace(0), stop_gradient=True,
       [[ 0.06601013,  0.37503667, -0.60469677,  0.68248436,  0.20264421],
        [-0.16129715,  0.02520340,  0.59349338,  0.51909989, -0.52911843]])
Tensor(shape=[2], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [0, 1])
Tensor(shape=[1], dtype=float64, place=CUDAPlace(0), stop_gradient=True,
       [1.76475810])
'''

Tensor(shape=[2, 5], dtype=float64, place=CUDAPlace(0), stop_gradient=True,
       [[ 0.06601013,  0.37503667, -0.60469677,  0.68248436,  0.20264421],
        [-0.16129715,  0.02520340,  0.59349338,  0.51909989, -0.52911843]])
Tensor(shape=[2], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [0, 1])
Tensor(shape=[1], dtype=float64, place=CUDAPlace(0), stop_gradient=True,
       [1.76475810])


In [104]:
input2=np.array([[ 0.06601013,  0.37503667, -0.60469677,  0.68248436,  0.20264421],[-0.16129715,  0.02520340,  0.59349338,  0.51909989, -0.52911843]])
labels2=np.array([0,1])
_softmax=np.exp(input2)/(np.sum(np.exp(input2),axis=1,keepdims=True))
print(f'自己计算softmax {_softmax}')
#_softmax_res=np.array([[0.17029278,0.23195555,0.0870787,0.31544787,0.19522511],[0.14287564,0.17216876,0.30392019,0.28213104,0.09890436]])
_softmax_paddle=F.softmax(paddle.to_tensor(input2))
print(f'paddle计算softmax {_softmax_paddle}')
log_res=paddle.log(_softmax_paddle)
print(log_res)
# softmax-log-NLLLoss
# 先求softmax，再求log，然后求NLLLoss
# NLLLoss的结果就是把上面的输出与Label对应的那个值拿出来，再去掉负号，再求均值。
# [Pytorch详解NLLLoss和CrossEntropyLoss](https://blog.csdn.net/qq_22210253/article/details/85229988)
print((-log_res[0][0]+-log_res[1][1])/2)


自己计算softmax [[0.17029278 0.23195555 0.0870787  0.31544787 0.19522511]
 [0.14287564 0.17216876 0.30392019 0.28213104 0.09890436]]
paddle计算softmax Tensor(shape=[2, 5], dtype=float64, place=CUDAPlace(0), stop_gradient=True,
       [[0.17029278, 0.23195555, 0.08707870, 0.31544787, 0.19522511],
        [0.14287564, 0.17216876, 0.30392019, 0.28213104, 0.09890436]])
Tensor(shape=[2, 5], dtype=float64, place=CUDAPlace(0), stop_gradient=True,
       [[-1.77023608, -1.46120954, -2.44094298, -1.15376185, -1.63360200],
        [-1.94578066, -1.75928011, -1.19099013, -1.26538362, -2.31360194]])
Tensor(shape=[1], dtype=float64, place=CUDAPlace(0), stop_gradient=True,
       [1.76475810])
